In [18]:
!pip install accelerate==0.30.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.2 MB/s eta 0:00:00


In [1]:
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

In [2]:
import json

file=json.load(open("config_file.json"))

In [3]:
hf_token=file["HF_TOKEN"]

Llama3 Model

In [4]:
model_name="meta-llama/Meta-Llama-3-8B"

Quantisation Configuration

In [6]:
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [7]:
tokenizer=AutoTokenizer.from_pretrained(model_name,token=hf_token)
tokenizer.pad_token=tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=hf_token
)


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [9]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128
)

In [10]:
def get_response(prompt):
  sequences = text_generator(prompt)
  gen_text = sequences[0]["generated_text"]
  return gen_text




In [15]:
text_generator("How many district in tamil nadu?")

[{'generated_text': 'How many district in tamil nadu? - Answers\nTamil Nadu has 32 districts. The districts are: Chennai, Thiruvallur, Kanchipuram, Dharmapuri, Dindigul, Erode, Karur, Krishnagiri, Madurai, Nagapattinam, Namakkal, Perambalur, Pudukottai, Salem, Sivaganga, Thanjavur, Theni, Thiruvarur, Thiruvannamalai, Thiruvarur, Thiruvarur, Thiruvarur, Thiruvarur, Thir'}]

In [21]:
text_generator("How many states in india?")

[{'generated_text': 'How many states in india? India is a federal republic made up of 28 states and 8 union territories. The union territories are directly ruled by the federal government, and the states are ruled by elected governments. The states are divided into districts, and the districts into tehsils or talukas.\nHow many states are there in India? India is a federal republic made up of 28 states and 8 union territories. The union territories are directly ruled by the federal government, and the states are ruled by elected governments. The states are divided into districts, and the districts into tehsils or talukas.'}]

In [22]:
usstate=text_generator("How many state are there in the USA")

In [26]:
usstate[0].get('generated_text').split("\n")[2]

'There are 50 states in the United States of America. They are Alabama, Alaska, Arizona, Arkansas, California, Colorado, Connecticut, Delaware, Florida, Georgia, Hawaii, Idaho, Illinois, Indiana, Iowa, Kansas, Kentucky, Louisiana, Maine, Maryland, Massachusetts, Michigan, Minnesota, Mississippi, Missouri, Montana, Nebraska, Nevada, New Hampshire, New Jersey, New Mexico, New York, North Carolina, North Dakota, Ohio, Oklahoma, Oregon, Pennsylvania, Rhode Island, South Carolina, South Dakota, Tennessee, Texas, Utah, Vermont, Virginia, Washington, West'